# Import Data

In [22]:
import pandas as pd
import cvxpy as cp
import numpy as np
import gdown

<IPython.core.display.Javascript object>

In [ ]:
url = "https://drive.google.com/uc?id=1TPXha3ojq2e9ygGN9r2249jPieaKL1ZR"
output = "flight_schedule.csv"
gdown.download(url, output)

In [4]:
df = pd.read_csv("flight_schedule.csv", index_col=0, header=1)
schedule = df.iloc[:-2]
cost = df.iloc[-2]
hours = df.iloc[-1]

<IPython.core.display.Javascript object>

In [5]:
cost

1    5
2    4
3    4
4    9
5    7
6    8
7    3
8    3
Name: Cost ($1000) for each sequence, dtype: int64

<IPython.core.display.Javascript object>

In [6]:
hours

1     396
2     352
3    1022
4     847
5     687
6     531
7     236
8     179
Name: Hours, dtype: int64

<IPython.core.display.Javascript object>

In [7]:
schedule = schedule.applymap(lambda x: 1 if x >= 1 else x)

<IPython.core.display.Javascript object>

In [8]:
schedule

,1,2,3,4,5,6,7,8
Requirement (Flight),,,,,,,,
New York to Buffalo,1,0,0,1,0,0,1,0
New York to Cincinnati,0,1,0,0,1,0,0,0
New York to Chicago,0,0,1,0,0,1,0,1
Buffalo to Chicago,1,0,0,1,0,0,0,0
Chicago to Cincinnati,0,0,1,1,0,1,0,0
Cincinnati to Pittsburgh,0,1,0,1,0,1,0,0
Cincinnati to Buffalo,0,0,1,0,1,0,0,0
Buffalo to New York,0,0,1,0,1,0,1,0
Pittsburgh to New York,0,1,0,1,0,1,0,0


<IPython.core.display.Javascript object>

# Minimize Cost

Input parameters: 
* $a$: 0-1 matrix
* $c_j$: cost for each feasible sequence of flight ($j=1, 2, ..8$)
* $b_i$: requirement vector ($i=1, 2, ..., 10$)

In [9]:
a = schedule.values
c = cost.values
b = np.ones(len(a))

<IPython.core.display.Javascript object>

Decision variable: 

$y_j=1$ or $0$: Whether the flight sequence $j$ is selected

In [10]:
y = cp.Variable(len(c), boolean=True)

<IPython.core.display.Javascript object>

Constraint: ensure that at least one crew is assigned to each flight.

For example, to make sure that at least one crew is assigned to the first flight, we have constraint:
$$y_1 + y_4 + y_7 \geq 1$$

Generalize this constraint for all 10 flights, we have:

$$
\begin{pmatrix}
  1  &  0  &  0  &  1  &  0  &  0  &  1  &  0 \\
  0  &  1  &  0  &  0  &  1  &  0  &  0  &  0 \\
  0  &  0  &  1  &  0  &  0  &  1  &  0  &  1 \\
  1  &  0  &  0  &  1  &  0  &  0  &  0  &  0 \\
  0  &  0  &  1  &  1  &  0  &  1  &  0  &  0 \\
  0  &  1  &  0  &  1  &  0  &  1  &  0  &  0 \\
  0  &  0  &  1  &  0  &  1  &  0  &  0  &  0 \\
  0  &  0  &  1  &  0  &  1  &  0  &  1  &  0 \\
  0  &  1  &  0  &  1  &  0  &  1  &  0  &  0 \\
  1  &  0  &  0  &  0  &  0  &  0  &  0  &  1 
\end{pmatrix}
\begin{pmatrix}
y_1\\
y_2\\
y_3\\
y_4\\
y_5\\
y_6\\
y_7\\
y_8\\
\end{pmatrix} \geq
\begin{pmatrix}
1\\
1\\
1\\
1\\
1\\
1\\
1\\
1\\
\end{pmatrix}
$$

$y_j=0$ or $1$ $j=1,2,...,8$

In [11]:
constraints = [a @ y >= b]

<IPython.core.display.Javascript object>

Objective: minimize the total cost of assigning crews to the selected sequence of flights

$$\min \;\; z=5y_1 + 4y_2 + 4y_3 + 9y_4 + 7y_5 + 8y_6 + 3y_7 + 3y_8$$

In [12]:
obj = cp.Minimize(c @ y)

<IPython.core.display.Javascript object>

In [13]:
prob = cp.Problem(obj, constraints)

<IPython.core.display.Javascript object>

In [14]:
prob.solve(solver=cp.GLPK_MI)

13.0

<IPython.core.display.Javascript object>

In [15]:
print(prob.status)

optimal


<IPython.core.display.Javascript object>

In [16]:
print(y.value)

[1. 1. 1. 0. 0. 0. 0. 0.]


<IPython.core.display.Javascript object>

# Hours

In [17]:
h = hours.values
h

array([ 396,  352, 1022,  847,  687,  531,  236,  179])

<IPython.core.display.Javascript object>

Constraints:
* There must be at least one airplane per flight
$$a \cdot y\geq b$$

* The total hours of flights must be less than or equal to 1700
$$h \cdot y \leq 1700$$

In [18]:
constraints = [a @ y >= b, h @ y <= 1700]

<IPython.core.display.Javascript object>

In [19]:
prob = cp.Problem(obj, constraints)

<IPython.core.display.Javascript object>

In [20]:
prob.solve(solver=cp.GLPK_MI)

20.0

<IPython.core.display.Javascript object>

In [21]:
print(y.value)

[1. 0. 0. 0. 1. 1. 0. 0.]


<IPython.core.display.Javascript object>

$$396 + 352 + 1022 = 1770$$
$$396 + 687 + 531 = 1614$$